In [9]:
import pandas as pd
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit



#initialize to use for later
wine_train_set=0
wine_test_set=0
#code is the same as from over on the initial exploration notebook.
dataframe = pd.read_csv("winequality-white.csv")
dataframeCopy = dataframe.copy()
dataframeCopy.dropna(axis='index', how='any', inplace=True)
dataframeCopy.dropna(axis='columns', how='any', inplace=True)

#stratifying data on quality to predict it
#reverted the test size
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=103)

for train_indices, test_indices in splitter.split(dataframeCopy, dataframeCopy["quality"]):
    wine_train_set = dataframeCopy.iloc[train_indices]
    wine_test_set = dataframeCopy.iloc[test_indices]
    

workingset=wine_train_set.copy()
testingset=wine_test_set.copy()

## initial set of features for X and the target feature y.

- My initial set of features for X are: residual sugar, alcohol, density,chlorides,volatile acidity, fixed acidity, sulphates, pH
 - The target feature for y is: quality 

- I chose these because in milestone one part b using these features I achieved an r-squared value of 0.2769342263179473, and a root mean squared error of: 0.7529132148619554 which were relatively close to the values of the training set, and overall for my dataset these values achieved the highest r-squared values, and root mean squared error values for the model.

In [15]:
#DECISION TREE FOR PART D-3 HERE
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold

y = workingset["quality"]
x = workingset[["residual sugar", "alcohol", "density","chlorides","volatile acidity", "fixed acidity", "sulphates", "pH"]]

tree_classifier = DecisionTreeClassifier()
tree_classifier.fit(x,y)

y_predicted = tree_classifier.predict(x)
matrix = confusion_matrix(y, y_predicted)
print(matrix)
print ("Accuracy is ", accuracy_score(y, y_predicted))

# We have to specify how to combine for the multiclassifications
print ("Precision is ", precision_score(y, y_predicted, average="weighted"))
print ("Sensitivity is ", recall_score(y, y_predicted, average="weighted"))
print ("F1 is ", f1_score(y, y_predicted, average="weighted"))

#results are likely overfitting so the below checks it.
# computed these metrics with the kFolds code from the lecutre material specifically because all of my confusion matrices were perfect and so was my accuracy and F-score.
validation_accuracy = []
validation_f1 =[]
fold_and_validate = KFold(n_splits=5, shuffle=True, random_state=145)
for train_set_indices, validation_set_indices in fold_and_validate.split(x):
    cv_train_set = x.iloc[train_set_indices]
    cv_train_target = y.iloc[train_set_indices]
    #print(cv_train_set)
    
    cv_decision_tree = DecisionTreeClassifier()
    cv_decision_tree.fit(cv_train_set, cv_train_target)
    
    cv_xvalidation = x.iloc[validation_set_indices]
    cv_y_true = y.iloc[validation_set_indices]
    cv_y_predicted = cv_decision_tree.predict(cv_xvalidation)
    
    cv_accuracy_score = accuracy_score(cv_y_true, cv_y_predicted)
    cv_f1_score = f1_score(cv_y_true, cv_y_predicted,  average="weighted")
    validation_accuracy.append(cv_accuracy_score)
    validation_f1.append(cv_f1_score)
    
print("Cross validation accuracies are: ", validation_accuracy)
print("Cross validation f1 scores  are: ", validation_f1)

[[  16    0    0    0    0    0    0]
 [   0  130    0    0    0    0    0]
 [   0    0 1166    0    0    0    0]
 [   0    0    0 1758    0    0    0]
 [   0    0    0    0  704    0    0]
 [   0    0    0    0    0  140    0]
 [   0    0    0    0    0    0    4]]
Accuracy is  1.0
Precision is  1.0
Sensitivity is  1.0
F1 is  1.0
Cross validation accuracies are:  [0.576530612244898, 0.5471938775510204, 0.5625, 0.5530012771392082, 0.5606641123882503]
Cross validation f1 scores  are:  [0.5793953210504539, 0.5479319719338513, 0.5618041410237935, 0.5570180226934941, 0.5606869706999108]


# Part 4-Thoughts on results

- Having perfect score for accuracy and other metrics with 1.0 would be great if it were not for the following: 
- It seems like to me that the data is likely overfitting because the confusion matrix is perfect with only one value per node for every node in this case so it doesn't seem like it's predicting anything.
- also per my in-class notes I see I am using more features than in the examples so I suspect that the amount of parameters I have is contributing to an overfit.
- To add to the first point about not predicting anything: the validation accuracies are fairly consistent and so too are the f1 scores but they differ greatly from the overall dataset's scores indicating that the model wouldn't do as great on unseen data, where as it's doing perfectly on the known data.